In [12]:
import pandas as pd
from xgboost import XGBClassifier

In [2]:
train_df = pd.read_csv("./data/train.csv")
train_df.head()

,id,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,...,feature_41,feature_42,feature_43,feature_44,feature_45,feature_46,feature_47,feature_48,feature_49,target
0,0,0,0,1,0,1,0,0,0,0,...,0,0,21,0,0,0,0,0,0,Class_2
1,1,0,0,0,0,2,1,0,0,0,...,0,0,0,0,0,0,0,0,0,Class_1
2,2,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,13,2,0,Class_1
3,3,0,0,0,0,0,0,0,3,0,...,0,0,0,0,0,0,0,1,0,Class_4
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,Class_2


In [3]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 52 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   id          100000 non-null  int64 
 1   feature_0   100000 non-null  int64 
 2   feature_1   100000 non-null  int64 
 3   feature_2   100000 non-null  int64 
 4   feature_3   100000 non-null  int64 
 5   feature_4   100000 non-null  int64 
 6   feature_5   100000 non-null  int64 
 7   feature_6   100000 non-null  int64 
 8   feature_7   100000 non-null  int64 
 9   feature_8   100000 non-null  int64 
 10  feature_9   100000 non-null  int64 
 11  feature_10  100000 non-null  int64 
 12  feature_11  100000 non-null  int64 
 13  feature_12  100000 non-null  int64 
 14  feature_13  100000 non-null  int64 
 15  feature_14  100000 non-null  int64 
 16  feature_15  100000 non-null  int64 
 17  feature_16  100000 non-null  int64 
 18  feature_17  100000 non-null  int64 
 19  feature_18  100000 non-n

In [4]:
for col in train_df.columns:
    print("% 25s" % col, " % 7d " % train_df[col].nunique(), train_df[col].unique()[:3])

                       id   100000  [0 1 2]
                feature_0       11  [0 1 9]
                feature_1       31  [0 1 6]
                feature_2        7  [1 0 3]
                feature_3       26  [0 1 2]
                feature_4       38  [1 2 0]
                feature_5       11  [0 1 2]
                feature_6       28  [0 3 2]
                feature_7       32  [ 0  3 13]
                feature_8       37  [ 0 15  1]
                feature_9       18  [0 2 6]
               feature_10       17  [ 0  2 10]
               feature_11       13  [0 1 3]
               feature_12       12  [0 1 2]
               feature_13        4  [0 1 2]
               feature_14       52  [0 1 7]
               feature_15       21  [ 0  3 11]
               feature_16       20  [0 1 9]
               feature_17       17  [6 3 0]
               feature_18       14  [0 3 4]
               feature_19       58  [ 0  1 28]
               feature_20       18  [0 2 3]
               fe

In [21]:
def estimate_n_iterations(pipeline, x_train, lbl_train, cols_used,
                          n_splits=2, groups_split=None,
                          nseed=1, verbose=True, fit_params=None):
    """
    :param pipeline: transformation pipeline. It has to be compatible with sklearn API
    :param x_train: training set
    :param y_train: training target
    :param n_splits: number of splits for crossvalidation
    :param n_jobs: number of CPUs to use in cross_validate function
    :param groups_split: columns to use in case of Groups split cv
    """

    
    cv = ShuffleSplit(n_splits=n_splits, test_size=0.3, random_state=0)
    
    x_train.drop(groups_split, axis=1, inplace=True)
    
    n_estimators = []
    scores = []
    for train_idx, valid_idx in cv.split(x_train, lbl_train):
        train_i = x_train.iloc[train_idx]
        valid_i = x_train.iloc[valid_idx]
        y_train_i = lbl_train[train_idx]
        y_valid_i = lbl_train[valid_idx]
        print(train_i.loc[:, cols_used].shape, valid_i.loc[:, cols_used].shape)
        #print(f"Train pos: %6d" %sum(y_train_i), ", neg: %7d" %(len(y_train_i) - sum(y_train_i)))
        #print(f"validation pos: %6d" %sum(y_valid_i), ", neg: %7d" %(len(y_valid_i) - sum(y_valid_i)))
        
        x_train_i = pipeline.fit_transform(train_i.loc[:, cols_used].copy(deep=True), y_train_i)
        x_valid_i = pipeline.transform(valid_i.loc[:, cols_used].copy(deep=True))
        
        est = XGBClassifier(**fit_params, use_label_encoder=False)
        
        est.fit(x_train_i, y_train_i,
                eval_set=[(x_train_i, y_train_i), (x_valid_i, y_valid_i)],
                eval_metric=["mlogloss"], verbose=500, early_stopping_rounds=50)
        
        n_estimators.append(est.best_iteration)
        scores.append(est.best_score)
        
    print("Score CV : ", np.round(np.mean(scores), 5), ' +/-', np.std(scores)) 
    print("Early stopping result : ", str(n_estimators))
    
    return n_estimators

In [29]:
params = {
    'n_estimators': 1500,
    'learning_rate': 0.03,
    'max_depth': 3,
    'subsample': 0.8,
    'colsample_bytree': 0.9,
    'n_jobs': 10,
    'tree_method': 'hist'
}

In [15]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import ShuffleSplit

from category_encoders.ordinal import OrdinalEncoder as OrdEncCE
from sklearn.preprocessing import OrdinalEncoder as OrdEncSKl

import numpy as np

In [16]:
cat_vars_1 = ['feature_1', 'feature_2', 'feature_3']
num_vars_1 = ['feature_4', 'feature_5', 'feature_6']

In [17]:
categorical_transformer = Pipeline(steps=[('Enc', OrdEncCE(handle_unknown="missing"))])
numerical_transformer = Pipeline(steps=[('Scaler', StandardScaler())])

preprocessor = ColumnTransformer(remainder='passthrough',
    transformers=[
        ('cat_vars_1', categorical_transformer, cat_vars_1),
        ('num_vars_1', numerical_transformer, num_vars_1)
    ])

pipeline = Pipeline(steps=[('preprocessing', preprocessor)])

In [18]:
target_encoder = OrdEncSKl().fit(train_df.target.values.reshape(-1, 1))

In [19]:
estimate_n_iterations(
    pipeline=pipeline,
    x_train=train_df.drop(['target'], axis=1).copy(),
    lbl_train=target_encoder.transform(train_df.target.values.reshape(-1, 1)),
    n_splits=4,
    groups_split='id',
    fit_params=params,
    cols_used=train_df.drop(['id', 'target'], axis=1).columns
)

(70000, 50) (30000, 50)
[0]	validation_0-mlogloss:1.38214	validation_1-mlogloss:1.38221
[300]	validation_0-mlogloss:1.09478	validation_1-mlogloss:1.10942
[600]	validation_0-mlogloss:1.07279	validation_1-mlogloss:1.10050
[900]	validation_0-mlogloss:1.05972	validation_1-mlogloss:1.09824
[1200]	validation_0-mlogloss:1.04949	validation_1-mlogloss:1.09704
[1499]	validation_0-mlogloss:1.04161	validation_1-mlogloss:1.09653
(70000, 50) (30000, 50)
[0]	validation_0-mlogloss:1.38212	validation_1-mlogloss:1.38222
[300]	validation_0-mlogloss:1.09204	validation_1-mlogloss:1.11370
[600]	validation_0-mlogloss:1.06977	validation_1-mlogloss:1.10563
[900]	validation_0-mlogloss:1.05638	validation_1-mlogloss:1.10338
[1200]	validation_0-mlogloss:1.04637	validation_1-mlogloss:1.10249
[1499]	validation_0-mlogloss:1.03872	validation_1-mlogloss:1.10206
(70000, 50) (30000, 50)
[0]	validation_0-mlogloss:1.38213	validation_1-mlogloss:1.38222
[300]	validation_0-mlogloss:1.09330	validation_1-mlogloss:1.11159
[600]	

[1497, 1498, 1499, 1398]

In [22]:
estimate_n_iterations(
    pipeline=pipeline,
    x_train=train_df.drop(['target'], axis=1).copy(),
    lbl_train=target_encoder.transform(train_df.target.values.reshape(-1, 1)),
    n_splits=4,
    groups_split='id',
    fit_params=params,
    cols_used=train_df.drop(['id', 'target'], axis=1).columns
)

(70000, 50) (30000, 50)
[0]	validation_0-mlogloss:1.37802	validation_1-mlogloss:1.37815
[500]	validation_0-mlogloss:1.05594	validation_1-mlogloss:1.09776
[855]	validation_0-mlogloss:1.03656	validation_1-mlogloss:1.09642
(70000, 50) (30000, 50)
[0]	validation_0-mlogloss:1.37795	validation_1-mlogloss:1.37818
[500]	validation_0-mlogloss:1.05269	validation_1-mlogloss:1.10295
[926]	validation_0-mlogloss:1.03036	validation_1-mlogloss:1.10196
(70000, 50) (30000, 50)
[0]	validation_0-mlogloss:1.37798	validation_1-mlogloss:1.37817
[500]	validation_0-mlogloss:1.05491	validation_1-mlogloss:1.10040
[918]	validation_0-mlogloss:1.03185	validation_1-mlogloss:1.09940
(70000, 50) (30000, 50)
[0]	validation_0-mlogloss:1.37805	validation_1-mlogloss:1.37813
[500]	validation_0-mlogloss:1.05447	validation_1-mlogloss:1.09811
[702]	validation_0-mlogloss:1.04230	validation_1-mlogloss:1.09790
Score CV :  1.09886  +/- 0.0020243627392342836
Early stopping result :  [805, 876, 868, 652]


[805, 876, 868, 652]

In [24]:
estimate_n_iterations(
    pipeline=pipeline,
    x_train=train_df.drop(['target'], axis=1).copy(),
    lbl_train=target_encoder.transform(train_df.target.values.reshape(-1, 1)),
    n_splits=4,
    groups_split='id',
    fit_params=params,
    cols_used=train_df.drop(['id', 'target'], axis=1).columns
)

(70000, 50) (30000, 50)
[0]	validation_0-mlogloss:1.37801	validation_1-mlogloss:1.37815
[500]	validation_0-mlogloss:1.05213	validation_1-mlogloss:1.09573
[932]	validation_0-mlogloss:1.02339	validation_1-mlogloss:1.09418
(70000, 50) (30000, 50)
[0]	validation_0-mlogloss:1.37798	validation_1-mlogloss:1.37821
[500]	validation_0-mlogloss:1.05016	validation_1-mlogloss:1.10130
[887]	validation_0-mlogloss:1.02435	validation_1-mlogloss:1.10011
(70000, 50) (30000, 50)
[0]	validation_0-mlogloss:1.37799	validation_1-mlogloss:1.37817
[500]	validation_0-mlogloss:1.05123	validation_1-mlogloss:1.09820
[822]	validation_0-mlogloss:1.02868	validation_1-mlogloss:1.09692
(70000, 50) (30000, 50)
[0]	validation_0-mlogloss:1.37803	validation_1-mlogloss:1.37812
[500]	validation_0-mlogloss:1.05113	validation_1-mlogloss:1.09623
[769]	validation_0-mlogloss:1.03174	validation_1-mlogloss:1.09570
Score CV :  1.09669  +/- 0.0021964477429477305
Early stopping result :  [882, 838, 772, 720]


[882, 838, 772, 720]

In [26]:
estimate_n_iterations(
    pipeline=pipeline,
    x_train=train_df.drop(['target'], axis=1).copy(),
    lbl_train=target_encoder.transform(train_df.target.values.reshape(-1, 1)),
    n_splits=4,
    groups_split='id',
    fit_params=params,
    cols_used=train_df.drop(['id', 'target'], axis=1).columns
)

(70000, 50) (30000, 50)
[0]	validation_0-mlogloss:1.37802	validation_1-mlogloss:1.37815
[500]	validation_0-mlogloss:1.05309	validation_1-mlogloss:1.09570
[946]	validation_0-mlogloss:1.02393	validation_1-mlogloss:1.09412
(70000, 50) (30000, 50)
[0]	validation_0-mlogloss:1.37799	validation_1-mlogloss:1.37821
[500]	validation_0-mlogloss:1.05101	validation_1-mlogloss:1.10129
[908]	validation_0-mlogloss:1.02439	validation_1-mlogloss:1.09994
(70000, 50) (30000, 50)
[0]	validation_0-mlogloss:1.37800	validation_1-mlogloss:1.37817
[500]	validation_0-mlogloss:1.05195	validation_1-mlogloss:1.09831
[906]	validation_0-mlogloss:1.02467	validation_1-mlogloss:1.09695
(70000, 50) (30000, 50)
[0]	validation_0-mlogloss:1.37803	validation_1-mlogloss:1.37812
[500]	validation_0-mlogloss:1.05199	validation_1-mlogloss:1.09613
[818]	validation_0-mlogloss:1.02986	validation_1-mlogloss:1.09542
Score CV :  1.09656  +/- 0.0021879600516233884
Early stopping result :  [896, 858, 857, 769]


[896, 858, 857, 769]

In [28]:
estimate_n_iterations(
    pipeline=pipeline,
    x_train=train_df.drop(['target'], axis=1).copy(),
    lbl_train=target_encoder.transform(train_df.target.values.reshape(-1, 1)),
    n_splits=4,
    groups_split='id',
    fit_params=params,
    cols_used=train_df.drop(['id', 'target'], axis=1).columns
)

(70000, 50) (30000, 50)
[0]	validation_0-mlogloss:1.37810	validation_1-mlogloss:1.37817
[500]	validation_0-mlogloss:1.07206	validation_1-mlogloss:1.09681
[1000]	validation_0-mlogloss:1.05072	validation_1-mlogloss:1.09383
[1114]	validation_0-mlogloss:1.04674	validation_1-mlogloss:1.09371
(70000, 50) (30000, 50)
[0]	validation_0-mlogloss:1.37807	validation_1-mlogloss:1.37825
[500]	validation_0-mlogloss:1.06982	validation_1-mlogloss:1.10226
[1000]	validation_0-mlogloss:1.04869	validation_1-mlogloss:1.09959
[1275]	validation_0-mlogloss:1.03955	validation_1-mlogloss:1.09941
(70000, 50) (30000, 50)
[0]	validation_0-mlogloss:1.37808	validation_1-mlogloss:1.37820
[500]	validation_0-mlogloss:1.07095	validation_1-mlogloss:1.09919
[1000]	validation_0-mlogloss:1.04944	validation_1-mlogloss:1.09667
[1203]	validation_0-mlogloss:1.04240	validation_1-mlogloss:1.09653
(70000, 50) (30000, 50)
[0]	validation_0-mlogloss:1.37810	validation_1-mlogloss:1.37816
[500]	validation_0-mlogloss:1.07137	validation_1

[1064, 1225, 1154, 860]

In [30]:
estimate_n_iterations(
    pipeline=pipeline,
    x_train=train_df.drop(['target'], axis=1).copy(),
    lbl_train=target_encoder.transform(train_df.target.values.reshape(-1, 1)),
    n_splits=4,
    groups_split='id',
    fit_params=params,
    cols_used=train_df.drop(['id', 'target'], axis=1).columns
)

(70000, 50) (30000, 50)
[0]	validation_0-mlogloss:1.37816	validation_1-mlogloss:1.37820
[500]	validation_0-mlogloss:1.08585	validation_1-mlogloss:1.09871
[1000]	validation_0-mlogloss:1.07204	validation_1-mlogloss:1.09423
[1499]	validation_0-mlogloss:1.06239	validation_1-mlogloss:1.09303
(70000, 50) (30000, 50)
[0]	validation_0-mlogloss:1.37812	validation_1-mlogloss:1.37828
[500]	validation_0-mlogloss:1.08348	validation_1-mlogloss:1.10423
[1000]	validation_0-mlogloss:1.06970	validation_1-mlogloss:1.10021
[1499]	validation_0-mlogloss:1.06018	validation_1-mlogloss:1.09926
(70000, 50) (30000, 50)
[0]	validation_0-mlogloss:1.37814	validation_1-mlogloss:1.37823
[500]	validation_0-mlogloss:1.08465	validation_1-mlogloss:1.10118
[1000]	validation_0-mlogloss:1.07080	validation_1-mlogloss:1.09713
[1499]	validation_0-mlogloss:1.06104	validation_1-mlogloss:1.09621
(70000, 50) (30000, 50)
[0]	validation_0-mlogloss:1.37815	validation_1-mlogloss:1.37820
[500]	validation_0-mlogloss:1.08555	validation_1

[1499, 1498, 1490, 1495]